In [ ]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import preprocessing
import gmplot
import matplotlib.pyplot as plt
from time import time
import operator
from datetime import datetime

In [ ]:
from geopy.geocoders import Nominatim
import overpass
import folium
from folium import plugins
from folium.plugins import HeatMap
geolocator = Nominatim(user_agent="TFG")

In [ ]:
# dataframe con device ids legibles
def device_ids_encoder(d):
    le = preprocessing.LabelEncoder()
    simple_ids=le.fit_transform(d['Device ID'].astype(str))    
    d['Device ID'] = simple_ids  #d['Device ID'].replace(simple_ids)   avoids warning, but too slow   
    return d

# punto medio de un usuario
def avg_location(d, user):
	j=0
	coords = [0, 0]
	for index, row in d.iterrows():
		if row['Device ID']==user:
			coords[0] += float(row['Latitude'])
			coords[1] += float(row['Longitude'])
			j += 1

	coords[0] = coords[0]/j;
	coords[1] = coords[1]/j;
	return coords

#Se obtiene un diccionario cuya clave es el user ID y el valor es un dataframe con las localizaciones del usuario
def get_info_by_user(d):
    users={}
    
    ids = list(d['Device ID'].unique())
    
    for x in ids:
        users[x] = d[d['Device ID'] == x]
        users[x] = users[x].sort_values(by='Time Stamp')
        users[x] = users[x].reset_index(drop=True)
            
    return users

# top n de usuarios por numero de localizaciones
def top_n_users(users, n):
	users_num_locations = {}
	users_top_n = {}
	i=0

	for k,v in users.items():
		users_num_locations[k] = len(v)

	users_num_locations = dict(sorted(users_num_locations.items(), key=operator.itemgetter(1), reverse=True))
    
	for k,v in users_num_locations.items():
		if i==n:
			break
		users_top_n[k] = users[k]
		i+=1
    
	return users_top_n

### Getting data from csv

In [ ]:
start_time = time()
#-----------------------------------------------------------------
# read csv
df_raw = pd.read_csv('balearesjulio.csv', sep = ',')
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

In [ ]:
df=df_raw
len(df_raw)

In [ ]:
df.insert(1, 'Date Time', df['Time Stamp'].apply(lambda x: datetime.fromtimestamp(x)))
df = df.sort_values(by='Time Stamp')
df = df.reset_index(drop=True)

In [ ]:
print("\nDataset original:")
df.head()

Número de usuarios distintos:

In [ ]:
len(df['Device ID'].unique())

In [ ]:
describe_table = df.describe()
describe_table.round(3)

### Creamos una caspeta y un csv para cada usuario

In [ ]:
users_top = []
users = list(df['Device ID'].unique())
for userID in users:
    if len(df[df['Device ID'] == userID]) > 850:
        users_top.append(userID)
len(users_top)

In [ ]:
start_time = time()
#-----------------------------------------------------------------
users = list(df['Device ID'].unique())
for userID in users_top:
    print(userID)
    df_user = df[df['Device ID'] == userID]
    df_user.to_csv("./users_Baleares/"+str(userID)+".csv", sep='\t', index=False)
    #df_user.drop(df_user.index, inplace=True)
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")